In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import os
import torch
import torchvision
import torchvision.transforms.functional as fn
from PIL import Image
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from models.models import LPCNNv1

from src.data.car_plate_dataset import UFPRDataset, id_to_filepath

C:\Users\Arya\anaconda3\envs\vision\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: -PKG-VERSION is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
os.chdir(r"C:/Users/Arya/workspace/ProjectSentry/data/raw/UFPR-ALPR dataset/testing/track0091")
os.getcwd()

'C:\\Users\\Arya\\workspace\\ProjectSentry\\data\\raw\\UFPR-ALPR dataset\\testing\\track0091'

In [101]:
train_dataset = UFPRDataset(
    "C:\\Users\\Arya\\workspace\\ProjectSentry\\data\\raw\\UFPR-ALPR dataset\\training",
    resize=torchvision.transforms.Resize(size=500),
    grayscale=torchvision.transforms.Grayscale()
)

finished setting up data


In [99]:
# train_dataset.labels.items()
# train_dataset.__getitem__(29)
# train_dataset.labels["000130"][1]

array([1152,  506, 1209,  531])

In [ ]:
train_dataloader = DataLoader(train_dataset)
trainer = pl.Trainer(max_epochs=1)
model = LPCNNv1()

trainer.fit(model, train_dataloaders=train_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type       | Params
--------------------------------------------
0 | conv1        | Conv2d     | 832   
1 | rel1         | ReLU       | 0     
2 | conv2        | Conv2d     | 51.3 K
3 | rel2         | ReLU       | 0     
4 | pool         | MaxPool2d  | 0     
5 | flatten      | Flatten    | 0     
6 | dense        | Linear     | 13.9 M
7 | rel3         | ReLU       | 0     
8 | bounding_box | Sequential | 772   
--------------------------------------------
13.9 M    Trainable params
0         Non-trainable params
13.9 M    Total params
55.631    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

torch.Size([1, 500, 888])
torch.Size([32, 496, 884])
torch.Size([64, 492, 880])
torch.Size([64, 246, 440])
torch.Size([64, 108240])
torch.Size([64, 128])
torch.Size([64, 4])
tensor([ 0.0300, -0.0491,  0.0155,  0.0197], grad_fn=<MeanBackward1>)
tensor([[421, 242, 455, 257]], dtype=torch.int32)
tensor(343.7459, grad_fn=<MeanBackward0>)
torch.Size([1, 500, 888])
torch.Size([32, 496, 884])
torch.Size([64, 492, 880])
torch.Size([64, 246, 440])
torch.Size([64, 108240])
torch.Size([64, 128])
torch.Size([64, 4])
tensor([ 0.1125, -0.0004,  0.0906,  0.0877], grad_fn=<MeanBackward1>)
tensor([[423, 242, 457, 256]], dtype=torch.int32)
tensor(344.4274, grad_fn=<MeanBackward0>)
torch.Size([1, 500, 888])
torch.Size([32, 496, 884])
torch.Size([64, 492, 880])
torch.Size([64, 246, 440])
torch.Size([64, 108240])
torch.Size([64, 128])
torch.Size([64, 4])
tensor([0.2609, 0.1255, 0.2343, 0.2268], grad_fn=<MeanBackward1>)
tensor([[425, 241, 460, 255]], dtype=torch.int32)
tensor(345.0381, grad_fn=<MeanBackward